In [ ]:
using Revise
using FUSE
using Plots
FUSE.logging(Logging.Info; actors=Logging.Info);
FUSE.ProgressMeter.ijulia_behavior(:clear);

In [ ]:
chk = FUSE.Checkpoint();
ini, act = FUSE.case_parameters(:D3D)
#ini.nb_unit[1].power_launched = t-> 2.5E6 + ramp(t-0.5) * 5E6
act.ActorHCD.nb_model = :RABBIT
dd = IMAS.dd();
FUSE.init(dd, ini, act);
chk[:init] = dd, ini, act;

In [ ]:
dd, ini, act = chk[:init];
plot(dd.core_sources)
FUSE.plot_plasma_overview(dd, dd.global_time; min_power=0.1E6, aggregate_radiation=true)

In [ ]:
dd, ini, act = chk[:init];
act.ActorCoreTransport.model = :FluxMatcher
FUSE.ActorStationaryPlasma(dd, act; verbose=true)
chk[:stationary] = dd, ini, act;

In [ ]:
dd, ini, act = chk[:stationary];
FUSE.plot_plasma_overview(dd, dd.global_time; min_power=0.1E6, aggregate_radiation=true)

In [ ]:
dd, ini, act = chk[:stationary];
act.ActorDynamicPlasma.Δt = 1.0
act.ActorDynamicPlasma.Nt = 15

act.ActorDynamicPlasma.evolve_current = false
act.ActorDynamicPlasma.evolve_equilibrium = false
act.ActorDynamicPlasma.evolve_pedestal = false
act.ActorDynamicPlasma.evolve_transport = false
act.ActorDynamicPlasma.evolve_hcd = true
act.ActorDynamicPlasma.evolve_pf_active = false
# act.ActorDynamicPlasma.ip_controller = false

FUSE.ActorDynamicPlasma(dd, act; verbose=true)
chk[:dynamic_no_transport] = dd, ini, act;

In [ ]:
dd, ini, act = chk[:dynamic_no_transport];
using Interact
@manipulate for time0 in dd.core_profiles.time[2:end]
    #plot(dd2.core_sources; time0)
    FUSE.plot_plasma_overview(dd, time0; min_power=0.1E6, aggregate_radiation=true)
end

In [ ]:
actor=FUSE.ActorRABBIT(dd,act);

In [ ]:
data = actor.outputs.powi_data
n=size(data)[2]
CList = reshape( range(colorant"red", stop=colorant"blue",length=n), 1, n );
plot(data;label="",seriescolor=CList)
#plot!(data[:,end];label="",seriescolor=CList)